<a href="https://colab.research.google.com/github/mJekal/PyTorch_study/blob/main/out_of_fold_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.model_selection import KFold

def train_apps_all_with_oof(apps_all_train, apps_all_test, nfolds=5):
    ftr_app = apps_all_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
    target_app = apps_all_train['TARGET']

    folds = KFold(n_splits=nfolds, shuffle=True, random_state=2020)

    oof_preds = np.zeros(ftr_app.shape[0])

    test_preds = np.zeros(apps_all_test.shape[0])

    clf = LGBMClassifier(
                nthread=4,
                n_estimators=4000,
                learning_rate=0.01,
                max_depth = 11,
                num_leaves=58,
                colsample_bytree=0.613,
                subsample=0.708,
                max_bin=407,
                reg_alpha=3.564,
                reg_lambda=4.930,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1,
                )

    for fold_idx, (train_idx, valid_idx) in enumerate(folds.split(ftr_app)):
        print('##### iteration ', fold_idx, ' 시작')
        train_x  = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200,
                early_stopping_rounds= 200)
        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        test_preds += clf.predict_proba(apps_all_test.drop('SK_ID_CURR', axis=1), num_iteration=clf.best_iteration_)[:, 1]/folds.n_splits


    return clf, test_preds

In [ ]:
clf, test_preds = train_apps_all_with_oof(apps_all_train, apps_all_test, nfolds=5)